In [10]:
import numpy as np
import pandas as pd
import pymysql
import os
from matplotlib import pyplot as plt
from pyecharts.charts import Bar,Line,Pie,Page,Grid
from pyecharts import options as opts
from pyecharts.commons.utils import JsCode
from pyecharts.globals import ChartType, SymbolType
plt.rcParams['font.sans-serif']=['SimHei']
plt.rcParams['axes.unicode_minus']=False

In [11]:
connection=pymysql.connect(
    host='localhost',
    user='root',
    password='123456',
    database='nysc'
)
try:
    with connection.cursor() as cursor:
        cursor.execute('SELECT * FROM agriculture')
        data_1 = cursor.fetchall()
    print('agriculture is getted')
finally:
    connection.close()

agriculture is getted


In [14]:
agriculture=pd.DataFrame(data_1,columns=['id','area','value','unit','zb','updatetime'])
agriculture.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4340 entries, 0 to 4339
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          4340 non-null   int64 
 1   area        4340 non-null   object
 2   value       4340 non-null   object
 3   unit        4340 non-null   object
 4   zb          4340 non-null   object
 5   updatetime  4340 non-null   object
dtypes: int64(1), object(5)
memory usage: 203.6+ KB


In [16]:
# 根据指标创建文件夹并保存数据
for indicator in agriculture['zb'].unique():
    # 创建以指标命名的文件夹
    indicator_folder = f"zb/{indicator}"
    if not os.path.exists(indicator_folder):
        os.makedirs(indicator_folder)

    # 获取该指标下的所有数据
    indicator_data = agriculture[agriculture['zb'] == indicator]

    # 根据年份进一步分组并保存
    for year in indicator_data['updatetime'].unique():
        grouped_data = indicator_data[indicator_data['updatetime'] == year]
        filename = f"{indicator_folder}/{year}.csv"  # 仅使用年份作为文件名
        grouped_data.to_csv(filename, index=False)

print("数据已保存")

数据已保存


1.农用化肥施用折纯量

In [5]:
# 1.2003-2022年农用化肥施用折纯量粮食产量

# 指定文件夹路径
folder_path = 'D:\\万陶哲\\毕业设计\\数据可视化\\zb\\农用化肥施用折纯量'
# 读取文件夹中的所有CSV文件
all_data = pd.DataFrame()
for file in os.listdir(folder_path):
    if file.endswith('.csv'):
        file_path = os.path.join(folder_path, file)
        data = pd.read_csv(file_path)
        all_data = pd.concat([all_data, data], ignore_index=True)

all_data['value'] = pd.to_numeric(all_data['value'])
# 将数据中值等于0的值删除
all_data = all_data[all_data['value'] != 0]

grouped_data = round(all_data.groupby('area')['value'].sum(),0).reset_index()
first_row=grouped_data.sort_values(by='value',ascending=False).iloc[0]
last_row=grouped_data.sort_values(by='value',ascending=False).iloc[-1]

new_Data=pd.DataFrame([first_row,last_row],columns=grouped_data.columns)
# new_Data

# 绘制柱状图
bar = (
    Bar()
    .add_xaxis(new_Data['area'].tolist())
    .add_yaxis("产量", new_Data['value'].sort_values(ascending=False).tolist(),label_opts=opts.LabelOpts(is_show=False))
    .set_global_opts(title_opts=opts.TitleOpts(title="全国农用化肥使用量对比"),yaxis_opts=opts.AxisOpts(name="单位(万吨)"))
)

bar.render("D:\\万陶哲\\毕业设计\\农业生产可视化系统\\application01\\templates\\zb\\农用化肥使用量bar.html")

'D:\\万陶哲\\毕业设计\\农业生产可视化系统\\application01\\templates\\zb\\农用化肥使用量bar.html'

2.粮食产量

In [6]:
# 2.2003-2022年粮食产量

# 指定文件夹路径
folder_path_1 = 'D:\\万陶哲\\毕业设计\\数据可视化\\zb\\粮食产量'
aut_all_data = pd.DataFrame()
for file in os.listdir(folder_path_1):
    if file.endswith('.csv'):
        file_path = os.path.join(folder_path_1, file)
        data = pd.read_csv(file_path)
        aut_all_data = pd.concat([aut_all_data, data], ignore_index=True)

aut_all_data['value'] = pd.to_numeric(aut_all_data['value'])
aut_all_data = aut_all_data[aut_all_data['value'] != 0]
grouped_aut_data = round(aut_all_data.groupby('updatetime')['value'].sum(),0).reset_index()

bar=(
    Bar()
    .add_xaxis(grouped_aut_data['updatetime'].tolist())
    .add_yaxis("产量", grouped_aut_data['value'].tolist(),label_opts=opts.LabelOpts(is_show=False),color='#FFA07A')
    .set_global_opts(title_opts=opts.TitleOpts(title="2003-2019年粮食产量变化"),yaxis_opts=opts.AxisOpts(name="单位(万吨)")
    )
)
bar.render("D:\\万陶哲\\毕业设计\\农业生产可视化系统\\application01\\templates\\zb\\粮食产量bar.html")

'D:\\万陶哲\\毕业设计\\农业生产可视化系统\\application01\\templates\\zb\\粮食产量bar.html'

3.播种面积

In [7]:
folder_path_2='D:\\万陶哲\\毕业设计\\数据可视化\\zb\\粮食播种面积'

area_aut=pd.DataFrame()
for file in os.listdir(folder_path_2):
    if file.endswith('.csv'):
        area_path=os.path.join(folder_path_2,file)
        data=pd.read_csv(area_path)
        area_aut=pd.concat([area_aut,data],ignore_index=True)

area_aut['value'] = pd.to_numeric(area_aut['value'])
area_aut = area_aut[area_aut['value'] != 0]
area_aut['updatetime'] = area_aut['updatetime'].astype(str)

group_area_1=round(area_aut.groupby('updatetime')['value'].sum(),0).reset_index()
line=(
    Line()
    .add_xaxis(group_area_1['updatetime'].tolist())
    .add_yaxis('粮食播种面积',group_area_1['value'].tolist(),is_smooth=True)
    .set_global_opts(title_opts=opts.TitleOpts(title="粮食播种面积变化"),yaxis_opts=opts.AxisOpts(name="单位(千公顷)")
    )
)
line.render('D:\\万陶哲\\毕业设计\\农业生产可视化系统\\application01\\templates\\zb\\粮食播种面积变化.html')

'D:\\万陶哲\\毕业设计\\农业生产可视化系统\\application01\\templates\\zb\\粮食播种面积变化.html'

4.受灾面积

In [8]:
folder_path_3='D:\\万陶哲\\毕业设计\\数据可视化\\zb\\受灾面积'

sz_area=pd.DataFrame()
for file in os.listdir(folder_path_3):
    if file.endswith('.csv'):
        area_path=os.path.join(folder_path_3,file)
        data=pd.read_csv(area_path)
        sz_area=pd.concat([sz_area,data],ignore_index=True)

sz_area = sz_area[sz_area['value'] != 0]
sorted_sz_area = sz_area.sort_values(by=['updatetime', 'value'], ascending=[True, False])

max_area_per_year = sorted_sz_area.loc[sorted_sz_area.groupby(sorted_sz_area['updatetime'].astype(str).str[:4])['value'].idxmax()]

area_counts = max_area_per_year['area'].value_counts()

sorted_area_counts = area_counts.sort_values(ascending=False)
sorted_area_counts_df = sorted_area_counts.reset_index()
sorted_area_counts_df.columns = ['area', 'count']
# 绘制柱状图
line = (
    Line()
    .add_xaxis(sorted_area_counts_df['area'].tolist())
    .add_yaxis("受灾次数", sorted_area_counts_df['count'].tolist(), label_opts=opts.LabelOpts(is_show=False))
    .set_global_opts(title_opts=opts.TitleOpts(title="近20年受灾面积最大省市"),yaxis_opts=opts.AxisOpts(name="单位(次数)"))
)

line.render("D:\\万陶哲\\毕业设计\\农业生产可视化系统\\application01\\templates\\zb\\受灾面积最大省市.html")

'D:\\万陶哲\\毕业设计\\农业生产可视化系统\\application01\\templates\\zb\\受灾面积最大省市.html'

5.有效灌溉面积

In [9]:
folder_path_4='D:\\万陶哲\\毕业设计\\数据可视化\\zb\\有效灌溉面积'

yxgg_area=pd.DataFrame()
for i in os.listdir(folder_path_4):
    if i.endswith('.csv'):
        file_path=os.path.join(folder_path_4,i)
        df=pd.read_csv(file_path)
        yxgg_area=pd.concat([yxgg_area,df],ignore_index=True)

yxgg_area = yxgg_area[yxgg_area['value'] != 0]
yxgg_area['value'] = pd.to_numeric(yxgg_area['value'])

yxgg_fifth_data_1=yxgg_area[yxgg_area['updatetime']==2003].sort_values(by='value',ascending=False).head(5)
yxgg_fifth_data_2=yxgg_area[yxgg_area['updatetime']==2007].sort_values(by='value',ascending=False).head(5)
yxgg_fifth_data_3=yxgg_area[yxgg_area['updatetime']==2012].sort_values(by='value',ascending=False).head(5)
yxgg_fifth_data_4=yxgg_area[yxgg_area['updatetime']==2021].sort_values(by='value',ascending=False).head(5)

pie_data_1 = yxgg_fifth_data_1[['area', 'value']].values.tolist()
pie_data_2 = yxgg_fifth_data_2[['area', 'value']].values.tolist()
pie_data_3 = yxgg_fifth_data_3[['area', 'value']].values.tolist()
pie_data_4 = yxgg_fifth_data_4[['area', 'value']].values.tolist()

pie1=(
    Pie()
    .add('',
          pie_data_1,radius=["45%", "75%"],label_opts=opts.LabelOpts(formatter="{b}: {c}千公顷"))
    .set_global_opts(title_opts=opts.TitleOpts(title="2003年有效灌溉面积前五"))
)
grid1 = (
    Grid()
    .add(pie1, grid_opts=opts.GridOpts(pos_left="10%", pos_top="20%"))
)

pie2=(
    Pie()
    .add('',
          pie_data_2,radius=["45%", "75%"],label_opts=opts.LabelOpts(formatter="{b}: {c}千公顷"))
    .set_global_opts(title_opts=opts.TitleOpts(title="2007年有效灌溉面积前五"))
)
grid2 = (
    Grid()
    .add(pie2, grid_opts=opts.GridOpts(pos_left="10%", pos_top="20%"))
)

pie3=(
    Pie()
    .add('',
          pie_data_3,radius=["45%", "75%"],label_opts=opts.LabelOpts(formatter="{b}: {c}千公顷"))
    .set_global_opts(title_opts=opts.TitleOpts(title="2012年有效灌溉面积前五"))
)
grid3 = (
    Grid()
    .add(pie3, grid_opts=opts.GridOpts(pos_left="10%", pos_top="20%"))
)

pie4=(
    Pie()
    .add('',
          pie_data_4,radius=["45%", "75%"],label_opts=opts.LabelOpts(formatter="{b}: {c}千公顷"))
    .set_global_opts(title_opts=opts.TitleOpts(title="2021年有效灌溉面积前五"))
)
grid4 = (
    Grid()
    .add(pie4, grid_opts=opts.GridOpts(pos_left="10%", pos_top="20%"))
)
page = Page(layout=Page.SimplePageLayout)
page.add(pie1, pie2,pie3,pie4)
page.render('D:\\万陶哲\\毕业设计\\农业生产可视化系统\\application01\\templates\\zb\\有效灌溉面积前五的省份Pie.html')


'D:\\万陶哲\\毕业设计\\农业生产可视化系统\\application01\\templates\\zb\\有效灌溉面积前五的省份Pie.html'